In [3]:
#import all necerssary libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium_stealth import stealth
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import time
import random
import csv
import os

Decided to scrap the glassdoor job board to get insights into the demand for Power Bi versus Tableau in the global analytics job market. The scrapper will pretty much work like the myjobmag scrapper, upon isnpection of the website's HTML structure, once you search a job keyword, say 'Business Intelligence Analyst', you get redirected to a results page populated with several job cards. To get access to the full job description of the job, you have to click on the job card, which dynamically loads a window on the right with the full details. 

In [4]:
#function to initialize the browser

def init_driver():
#set up the browser using the options object

    options=webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option('excludeSwitches',['enable_automation'])
    options.add_experimental_option('useAutomationExtension','False')

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
)   
    stealth (driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )
    return driver



In [5]:
#adding the resume feature, if a job has already been scrapped, the scrapper will skip it

scraped_links = set()

if os.path.exists("glassdoor_data.csv"):
    with open("glassdoor_data.csv", "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            scraped_links.add(row["link"])

print(f"Resuming... {len(scraped_links)} jobs already scraped.")

# append mode, so as not to overide data already in the csv
csv_file = open("glassdoor_data.csv", "a", newline="", encoding="utf-8")
writer = csv.DictWriter(csv_file, fieldnames=["title", "description", "link"])

# Write header only if file is empty
if os.stat("glassdoor_data.csv").st_size == 0:
    writer.writeheader()

Resuming... 0 jobs already scraped.


In [10]:
#initialize the driver
driver=init_driver()
base_url=('https://www.glassdoor.com/Job/jobs.htm?sc.occupationParam=power+bi%2C+tableau&sc.locationSeoString=United+States&locId=1&locT=N')
time.sleep(random.uniform(4,6))

counter=0
job_links=[]
for count in range(1,2): #scrap for 50 loops only
    driver.get(base_url)
    time.sleep(random.uniform(4,6))
    link_elements = driver.find_elements(By.CLASS_NAME, "JobCard_jobTitle")
    for element in link_elements:
        href=element.get_attribute('href')
        job_links.append(href)






[]